# Smart-G demo notebook for Objects

This is an interactive document allowing to run Smart-G with python and visualize the results. <br>
*Tips*: cells can be executed with shift-enter. Tooltips can be obtained with shift-tab. More information [here](http://ipython.org/notebook.html) or in the help menu. [A table of content can also be added](https://github.com/minrk/ipython_extensions#table-of-contents).

In [ ]:
%pylab inline
# next 2 lines allow to automatically reload modules that have been changed externally
%reload_ext autoreload
%autoreload 2

import os, sys, re
sys.path.insert(0, os.path.dirname(os.getcwd()))
from smartg.smartg import Smartg
from smartg.smartg import LambSurface
from smartg.atmosphere import AtmAFGL, AeroOPAC

import smartg.geometry
from smartg.geometry import Point, Vector
import smartg.visualizegeo
from smartg.visualizegeo import receiver_view, cat_view, Mirror, Plane, Transformation, \
    Entity, Analyse_create_entity, Matte, generateHfP, generateHfA, extractPoints
from smartg.smartg import CusForward, Sensor
from luts.luts import Idx
from smartg.tools.smartg_view import smartg_view

In [ ]:
# Uncomment below for 3D views in (only) jupyter lab
# %matplotlib widget 
# import matplotlib.pyplot as plt

# Quick Start

## Creation of objects (simple example)

In [ ]:
# We want to create a simple case with a receiver and four heliostats (MirA, B, C and D).
# The receiver will be created a x = 1km, the first heliostat at x = 950m, the
# second heliostat at x = 900m and so on. The mirror will be oriented such that the
# direct rays reflected by the mirrors will totally reach the receiver.

# The solar zenith angle
solarDir = 14.3
# Create the heliostats MirA, MirB, MirC and MirD (smartg unit is the kilometer)
wMx = 0.004725
wMy = 0.00642
MirA = Entity(name = "reflector", \
              materialAV = Mirror(reflectivity = 0.88), \
              materialAR = Matte(), \
              geo = Plane( p1 = Point(-wMx, -wMy, 0.),
                           p2 = Point(wMx, -wMy, 0.),
                           p3 = Point(-wMx, wMy, 0.),
                           p4 = Point(wMx, wMy, 0.) ), \
              transformation = Transformation( rotation = np.array([0., 20.281725, 0.]), \
                                               translation = np.array([-0.05, 0., 0.00517]) ))

MirB = Entity(MirA); MirC = Entity(MirA); MirD = Entity(MirA);
MirB.transformation = Transformation( rotation = np.array([0., 29.460753, 0.]), \
                                      translation = np.array([-0.1, 0., 0.00517]) )
MirC.transformation = Transformation( rotation = np.array([0., 35.129831, 0.]), \
                                      translation = np.array([-0.15, 0., 0.00517]) )
MirD.transformation = Transformation( rotation = np.array([0., 38.715473, 0.]), \
                                      translation = np.array([-0.2, 0., 0.00517]) )

# Create the receiver Recept1
wRx = 0.006
wRy = 0.007
# TC = Taille cellule. The receiver is devided in several cells to visualize the flux distribution 
Recept1 = Entity(name = "receiver", TC = 0.0005, \
                 materialAV = Matte(reflectivity = 0.), \
                 materialAR = Matte(reflectivity = 0.), \
                 geo = Plane( p1 = Point(-wRx, -wRy, 0.),
                              p2 = Point(wRx, -wRy, 0.),
                              p3 = Point(-wRx, wRy, 0.),
                              p4 = Point(wRx, wRy, 0.) ), \
                 transformation = Transformation( rotation = np.array([0., -101.5, 0.]), \
                                                  translation = np.array([0., 0., 0.1065]) ))

# Create the list containing all the objects
listobjs = [MirA, MirB, MirC, MirD, Recept1]

## How to verify the drawing of objects

In [ ]:
# 3D print of objects (the notebook give only a 2D print of the 3D one)
fig1 = Analyse_create_entity(ENTITY = listobjs, THEDEG = solarDir)

## Run the simulation

In [ ]:
# In this simulation the photons from TOA are launched to fill only the
# mirrors (ray tracing method or restricted forward method) --> cusForward = CusForward(LMODE="RF")
# By default the sun is a ponctual sun source targeting the origin (0,0,0) --> --> cusForward = None
# For a full forward mode i.g. specify in Smartg.run() -> cusForward = CusForward(CFX=10, CFY=10, LMODE="FF")
# Where CFX is the size in kilometer in the x axis of the rectangle. Be careful, the full forward mode 
# need a big number of photons to obtain a good accuracy !

w2 = 0.5
Pmin = [-w2, -w2, -0.005]
Pmax = [w2, w2, 0.125]
interval0 = [Pmin, Pmax] # enable to earn some computational time, can be very usefull in FF mode
aer = AeroOPAC('desert', 0.25, 550.)
pro = AtmAFGL('afglms', comp=[aer], P0 = 877, H2O=1.2)
# pro = AtmAFGL('afglms', P0 = 877, H2O=1.2)
m = Smartg(double = True, obj3D = True).run( surf = LambSurface(ALB=0.25), 
    THVDEG=solarDir, NF=1e6, wl=550., NBPHOTONS=4e7, NBLOOP = 5e6, atm=pro,
    myObjects=listobjs, interval = interval0, cusL = CusForward(LMODE="RF"))

## How to show the results ?

In [ ]:
# Show the description of the simulation
m.print_info(show_attrs=True)

In [ ]:
# print the infomation at the receiver
cat_view(m)

In [ ]:
# print the flux distribution at the receiver
receiver_view(m['C_Receiver'][:,:], [wRx, wRy], logI=False)

In [ ]:
# print the flux distribution at the receiver from only the direct D (cat1)
receiver_view(m['C1_Receiver'][:,:], [wRx, wRy], logI=False)

In [ ]:
# print the flux distribution at the receiver from beams from the process H only (cat2)
receiver_view(m['C2_Receiver'][:,:], [wRx, wRy], logI=False)

In [ ]:
# print the flux distribution at the receiver from beams from the process E only (cat3)
receiver_view(m['C3_Receiver'][:,:], [wRx, wRy], logI=False)

In [ ]:
# print the flux distribution at the receiver from beams from the process A only (cat4)
receiver_view(m['C4_Receiver'][:,:], [wRx, wRy], logI=False)

In [ ]:
# print the flux distribution at the receiver from beams from the two processes H and A (cat5)
receiver_view(m['C5_Receiver'][:,:], [wRx, wRy], logI=True)

In [ ]:
# print the flux distribution at the receiver from beams from the two processes H and E (cat6)
receiver_view(m['C6_Receiver'][:,:], [wRx, wRy], logI=False)

In [ ]:
# print the flux distribution at the receiver from beams from the two processes E and A (cat7)
receiver_view(m['C7_Receiver'][:,:], [wRx, wRy], logI=False)

In [ ]:
# print the flux distribution at the receiver from beams from the three processes H , E and A (cat8)
receiver_view(m['C8_Receiver'][:,:], [wRx, wRy], logI=False)

# More complex cases

## Quick heliostat generation by giving two angles

In [ ]:
# Specify the solar zenith angle
solarDir = 14.3

# Creation of a receiver
wRx = 0.006
wRy = 0.007

Recept2 = Entity(name = "receiver", TC = 0.0005, \
                 materialAV = Matte(reflectivity = 0.), \
                 materialAR = Matte(reflectivity = 0.), \
                 geo = Plane( p1 = Point(-wRx, -wRy, 0.),
                              p2 = Point(wRx, -wRy, 0.),
                              p3 = Point(-wRx, wRy, 0.),
                              p4 = Point(wRx, wRy, 0.) ), \
                 transformation = Transformation( rotation = np.array([0., -101.5, 0.]), \
                                                  translation = np.array([0., 0., 0.1065]) ))

pRe = Point(Recept2.transformation.transx, Recept2.transformation.transy, Recept2.transformation.transz)

# Generation of heliostats thanks to two angles, MINANG and MAXANG.
listobjs1 = generateHfA(THEDEG=solarDir, PR=pRe, MINANG=150, MAXANG=210, GAPDEG = 10,
                        FDRH=0.1, NBH = 3, GAPDIST = 0.008, HSX = 0.005, HSY = 0.01,
                        PILLH= 0.00517, REF=0.88)

# Without forgetting to add the reveicer at the list of objects
listobjs1.append(Recept2)

## Drawing verification

In [ ]:
fig1 = Analyse_create_entity(listobjs1, THEDEG = solarDir)

## Run the simulation

In [ ]:
aer = AeroOPAC('desert', 1, 550.)
pro = AtmAFGL('afglms', comp=[aer])
lMode1 = CusForward(LMODE="RF")

m2 = Smartg(double = True, obj3D = True).run( surf = LambSurface(ALB=0.25), 
        THVDEG=solarDir, NF=1e6, wl=550., NBPHOTONS=1e7, NBLOOP = 1e6, atm=pro,
        myObjects=listobjs1, cusL = lMode1)

In [ ]:
m2.print_info(show_attrs=True)

## Results

In [ ]:
cat_view(m2, acc=3)

In [ ]:
receiver_view(m2['C_Receiver'][:,:], [wRx, wRy], logI=True)

## Quick heliostat generation by giving positions from a file

In [ ]:
# You need a file with the x, y and z positions, see the file HPOS_STP1.dat as example
# Points of heliostats are extracted from the given file as a list of class point
lPH = extractPoints(filename = "../auxdata/STPs/STP1.dat")

# Specify the solar zenith angle
solarDir = 14.3

# Creation of a receiver
wRx = 0.006
wRy = 0.007

Recept3 = Entity(name = "receiver", TC = 0.0005, \
                 materialAV = Matte(reflectivity = 0.), \
                 materialAR = Matte(reflectivity = 0.), \
                 geo = Plane( p1 = Point(-wRx, -wRy, 0.),
                              p2 = Point(wRx, -wRy, 0.),
                              p3 = Point(-wRx, wRy, 0.),
                              p4 = Point(wRx, wRy, 0.) ), \
                 transformation = Transformation( rotation = np.array([0., -101.5, 0.]), \
                                                  translation = np.array([0., 0., 0.1065]) ))

# Coordinate of the center of the created receiver needed for heliostat generation
pRe = Point(Recept3.transformation.transx, Recept3.transformation.transy, Recept3.transformation.transz)

# Generate heliostats thanks to a list of Points, lPH.
listobjs2 = generateHfP(THEDEG = 14.3, PH = lPH, PR = pRe,
                        HSX = 0.00945, HSY = 0.01284, REF = 0.88)

# Without forgetting to add the reveicer at the list of objects
listobjs2.append(Recept3)

## Drawing verification

In [ ]:
fig1 = Analyse_create_entity(listobjs2, THEDEG = solarDir)

## Run the simulation

In [ ]:
Pmin = [-0.12, -0.05, -0.05]
Pmax = [0.05, 0.05, 0.125]
# Pmin = [-120, -120, -0.05]
# Pmax = [120, 120, 121]
interval0 = [Pmin, Pmax] # enable to earn some computational time, can be very usefull in FF mode
aer = AeroOPAC('desert', 0.2, 550.)
pro = AtmAFGL('afglms', comp=[aer])
lMode = CusForward(CFX = 0.06, CFY = 0.08, CFTX = -0.08, CFTY = 0., LMODE="FF")
m3 = Smartg(double = True, obj3D = True).run( surf = LambSurface(ALB=0.25), 
        THVDEG=solarDir, NF=1e6, wl=550., NBPHOTONS=4e7, NBLOOP = 2e6, atm=pro,
        myObjects=listobjs2, interval = interval0, cusL = lMode)

In [ ]:
m3.print_info(show_attrs=True)

## Results

In [ ]:
cat_view(m3)

In [ ]:
receiver_view(m3['C_Receiver'][:,:], [wRx, wRy], logI=False, vmin=0)

# Backward mode with objects

## Creation of the objects

In [ ]:
# Zenith and Azimuth angle of the sun, respectively Theta and phi
Theta = 50
Phi   = 45

# Position of heliostats (list) [pH] and position of the sensor pS
pH = [Point(-0.05, 0., 0.00517)] # here only one
pS = Point(0., 0., 0.1065)

# create the heliostats : here we need the zenith and azimuth angles
objs = generateHfP(THEDEG = Theta, PHIDEG = Phi, PH = pH, PR = pS, 
                   HSX = 0.00945, HSY = 0.01284, REF = 0.88)

# Modify the rugosity ? here -->
objs[0].materialAV = Mirror(reflectivity = 0.88, roughness=0.1, shadow=True)

# Creation of the sensor
vS = Vector(pH[0]-pS) # direction of the sensor described by a vector
SEN = Sensor(POSX = pS.x, POSY = pS.y, POSZ = pS.z, LOC  = 'ATMOS', FOV  = 0., TYPE = 0, V=vS)

## Run the simulation

In [ ]:
aer = AeroOPAC('desert', 0.2, 550.)
pro = AtmAFGL('afglms', comp=[aer])
m4 = Smartg(double = True, obj3D = True, back = True).run( surf = LambSurface(ALB=0.25),
                NF=1e6, wl=550., NBPHOTONS=1e8, NBLOOP = 1e7, atm=pro,
                myObjects=objs, sensor = SEN)

## See the results

In [ ]:
_=smartg_view(m4, QU=True, ind=Idx([0, 22, 44, 68, 90, 112, 134, 156]))